# Feature Extraction

This notebook demonstrates the capabilities of the toolkit focusing on transforming raw time-series data into meaningful features for machine learning applications.

## 📦 Importing Required Libraries

Let's start by importing all the necessary libraries and modules for feature extraction:


In [4]:
import pandas as pd
from pathlib import Path

# Core toolkit imports
from ThreeWToolkit.dataset import ParquetDataset
from ThreeWToolkit.core.base_dataset import ParquetDatasetConfig, EventPrefixEnum

# Preprocessing imports (needed for windowing)
from ThreeWToolkit.preprocessing import Windowing
from ThreeWToolkit.core.base_preprocessing import WindowingConfig

# Feature extraction imports
from ThreeWToolkit.feature_extraction import (
    ExtractStatisticalFeatures,
    ExtractWaveletFeatures, 
    ExtractEWStatisticalFeatures
)
from ThreeWToolkit.core.base_feature_extractor import (
    StatisticalConfig,
    WaveletConfig,
    EWStatisticalConfig
)

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


## 📊 Loading and Preparing the Dataset

Let's load the 3W dataset and prepare it for feature extraction. We'll use a subset of the data to make the examples more manageable:


In [5]:
# Define dataset path (adjust this path according to your setup)
dataset_path = Path("../../dataset")

# Configure dataset loading (only real events)
ds_config = ParquetDatasetConfig(
    path=dataset_path,
    clean_data=True,
    event_type=[EventPrefixEnum.REAL]  
)

# Load the dataset
ds = ParquetDataset(ds_config)
print(f"📁 Dataset loaded successfully!")
print(f"📊 Total number of events: {len(ds)}")


[ParquetDataset] Dataset found at ../../dataset
[ParquetDataset] Validating dataset integrity...
[ParquetDataset] Dataset integrity check passed!
📁 Dataset loaded successfully!
📊 Total number of events: 1119


Let's select any event and its labels:

In [16]:
id_event = 480

x_raw, y_raw = ds[id_event]["signal"], ds[id_event]["label"]
x_raw.head()


,ABER-CKGL,ABER-CKP,ESTADO-DHSV,ESTADO-M1,ESTADO-M2,ESTADO-PXO,ESTADO-SDV-GL,ESTADO-SDV-P,ESTADO-W1,ESTADO-W2,...,P-JUS-CKGL,P-JUS-CKP,P-MON-CKP,P-PDG,P-TPT,QGL,T-JUS-CKP,T-MON-CKP,T-PDG,T-TPT
timestamp,,,,,,,,,,,,,,,,,,,,,
2014-01-24 08:33:03,0.0,0.0,0.867921,0.414652,-0.681653,-0.094347,-1.094009,0.312558,0.650525,-0.563169,...,0.0,0.0,1.896432,0.0,0.756475,-0.846093,0.290959,0.0,0.0,0.654069
2014-01-24 08:33:04,0.0,0.0,0.867921,0.414652,-0.681653,-0.094347,-1.094009,0.312558,0.650525,-0.563169,...,0.0,0.0,1.896433,0.0,0.756475,-0.846093,0.290959,0.0,0.0,0.654092
2014-01-24 08:33:05,0.0,0.0,0.867921,0.414652,-0.681653,-0.094347,-1.094009,0.312558,0.650525,-0.563169,...,0.0,0.0,1.896433,0.0,0.756475,-0.846093,0.290959,0.0,0.0,0.654111
2014-01-24 08:33:06,0.0,0.0,0.867921,0.414652,-0.681653,-0.094347,-1.094009,0.312558,0.650525,-0.563169,...,0.0,0.0,1.896433,0.0,0.756475,-0.846093,0.290959,0.0,0.0,0.654134
2014-01-24 08:33:07,0.0,0.0,0.867921,0.414652,-0.681653,-0.094347,-1.094009,0.312558,0.650525,-0.563169,...,0.0,0.0,1.896433,0.0,0.756475,-0.846093,0.290959,0.0,0.0,0.654154


-----------

## 🌊 Wavelet Feature Extraction


The `ExtractWaveletFeatures` class applies the **Stationary Wavelet Transform (SWT)**, a signal processing method that decomposes each window of the signal into multiple frequency components. This approach can reveal patterns that are often hidden from conventional statistical methods.

At each decomposition level, SWT produces two types of coefficients:

- **Approximation Coefficients (A)**: Represent the low-frequency content of the signal, capturing its general trend or smooth behavior within the window. You can think of these as a "smoothed" version of the original signal.

- **Detail Coefficients (D)**: Represent the high-frequency content, including noise, spikes, and abrupt changes. These features help the model distinguish between the overall signal shape and its fine-grained, high-frequency variations.

In [17]:
# Define parameters that will be shared between windowing and feature extraction
# The window_size for the wavelet transform is determined by its level
LEVEL = 3
WINDOW_SIZE = 2**LEVEL  
OVERLAP = 0.875       # This is equivalent to a stride of 1 for a window of 8

print(f"🔧 Windowing Configuration:")
print(f"   Window size: {WINDOW_SIZE}")
print(f"   Overlap: {OVERLAP} ({OVERLAP*100}%)")
print(f"   Stride: {int(WINDOW_SIZE * (1 - OVERLAP))}")

# Configure and instantiate the Windowing step
# We use a 'boxcar' window because we don't want to change the signal
# before applying the wavelet transform.
windowing_config = WindowingConfig(
    window_size=WINDOW_SIZE,
    overlap=OVERLAP,
    window="boxcar" 
)
windowing_step = Windowing(windowing_config)

# Run the windowing step on the raw data
windowed_x = windowing_step(x_raw)

# Align the labels (y)
# The Windowing class doesn't handle labels, so we must align them manually.
# The label for each window corresponds to the label at the end of that window in the original series.
step = int(WINDOW_SIZE * (1 - OVERLAP))
if step == 0: 
    step = 1
end_of_window_indices = [
    i + WINDOW_SIZE - 1 for i in range(0, len(x_raw) - WINDOW_SIZE + 1, step)
]
aligned_y = y_raw.iloc[end_of_window_indices]

# Combine the windowed features and aligned labels into a single DataFrame
#  We reset the index to ensure they concatenate correctly.
windowed_data = pd.concat([
    windowed_x.reset_index(drop=True),
    aligned_y.reset_index(drop=True)
], axis=1)

print(f"\n✅ Windowing completed!")
print(f"   Original data shape: {X_raw.shape}")
print(f"   Windowed data shape: {windowed_data.shape}")
print(f"   Number of windows created: {len(windowed_data)}")

# Display the first few rows of windowed data
print(f"\n📋 First few rows of windowed data:")
windowed_data.head()


🔧 Windowing Configuration:
   Window size: 8
   Overlap: 0.875 (87.5%)
   Stride: 1

✅ Windowing completed!
   Original data shape: (154801, 22)
   Windowed data shape: (62061, 178)
   Number of windows created: 62061

📋 First few rows of windowed data:


,var1_t0,var1_t1,var1_t2,var1_t3,var1_t4,var1_t5,var1_t6,var1_t7,var2_t0,var2_t1,...,var22_t0,var22_t1,var22_t2,var22_t3,var22_t4,var22_t5,var22_t6,var22_t7,win,class
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.654069,0.654092,0.654111,0.654134,0.654154,0.654177,0.654197,0.654219,1,1
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.654092,0.654111,0.654134,0.654154,0.654177,0.654197,0.654219,0.654239,2,1
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.654111,0.654134,0.654154,0.654177,0.654197,0.654219,0.654239,0.654262,3,1
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.654134,0.654154,0.654177,0.654197,0.654219,0.654239,0.654262,0.654282,4,1
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.654154,0.654177,0.654197,0.654219,0.654239,0.654262,0.654282,0.654305,5,1


In [18]:
# Configure the feature extractor
wavelet_config = WaveletConfig(
    level=LEVEL, 
    wavelet="haar",
)

# Instantiate the extractor
feature_extractor = ExtractWaveletFeatures(wavelet_config)
feature_extractor.is_windowed=True

# Run the feature extraction step by calling the instance
wavelet_features = feature_extractor(windowed_data)

# Display the results
print("Shape of the final extracted features:", wavelet_features.shape)
print("\nColumns are named 'var<index>_<feature>', and the 'label' column is preserved.")
wavelet_features.head()


Shape of the final extracted features: (62061, 154)

Columns are named 'var<index>_<feature>', and the 'label' column is preserved.


,var1_A3,var1_D3,var1_A2,var1_D2,var1_A1,var1_D1,var1_A0,var2_A3,var2_D3,var2_A2,...,var21_A1,var21_D1,var21_A0,var22_A3,var22_D3,var22_A2,var22_D2,var22_A1,var22_D1,var22_A0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.850199,0.00012,1.308373,0.000043,0.92519,0.000016,0.654219
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.850259,0.00012,1.308416,0.000043,0.92522,0.000014,0.654239
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.850320,0.00012,1.308459,0.000043,0.92525,0.000016,0.654262
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.850380,0.00012,1.308501,0.000043,0.92528,0.000014,0.654282
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.850440,0.00012,1.308544,0.000043,0.92531,0.000016,0.654305


In [19]:
# Configure with an offset
offset = 20
config_offset = WaveletConfig(level=LEVEL, overlap=0.875, offset=offset)
extractor_offset = ExtractWaveletFeatures(config_offset)
extractor_offset.is_windowed = True
features_offset = extractor_offset(windowed_data)

features_offset.head()


,var1_A3,var1_D3,var1_A2,var1_D2,var1_A1,var1_D1,var1_A0,var2_A3,var2_D3,var2_A2,...,var21_A1,var21_D1,var21_A0,var22_A3,var22_D3,var22_A2,var22_D2,var22_A1,var22_D1,var22_A0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.851407,0.000122,1.309228,0.000043,0.925794,0.000014,0.654645
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.851467,0.000122,1.309271,0.000043,0.925824,0.000016,0.654668
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.851527,0.000122,1.309313,0.000043,0.925855,0.000014,0.654688
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.851589,0.000120,1.309356,0.000043,0.925885,0.000016,0.654711
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.851649,0.000120,1.309399,0.000043,0.925915,0.000014,0.654730


-------

## 📊 Statistical Feature Extraction

This is the most common approach to feature extraction for time-series data. The `ExtractStatisticalFeatures` class takes the pre-windowed data and calculates a set of standard statistical descriptors for each window. These features summarize the shape and distribution of the data within that specific time segment.

The features extracted are:
* **`mean`, `std`**: Describe the central tendency and dispersion (volatility).
* **`skew`, `kurtosis`**: Describe the shape of the distribution (asymmetry and presence of outliers).
* **`min`, `1qrt`, `med`, `3qrt`, `max`**: Provide a summary of the distribution through quartiles.

Since our data is already windowed from the previous step, we can reuse the `windowed_data` `DataFrame` directly.


In [20]:
# Configure the statistical feature extractor
statistical_config = StatisticalConfig(
    window_size=WINDOW_SIZE,
    overlap=OVERLAP,
)

# Instantiate the extractor
statistical_feature_extractor = ExtractStatisticalFeatures(statistical_config)
statistical_feature_extractor.is_windowed = True

# Run the feature extraction step by calling the instance
statistical_features = statistical_feature_extractor(windowed_data)

# Display the results
print("Shape of the final statistical features:", statistical_features.shape)
print("\nColumns are named 'var<index>_<feature>', and the label column is preserved.")
statistical_features.head()


Shape of the final statistical features: (62061, 198)

Columns are named 'var<index>_<feature>', and the label column is preserved.


,var1_mean,var1_std,var1_skew,var1_kurt,var1_min,var1_1qrt,var1_med,var1_3qrt,var1_max,var2_mean,...,var21_max,var22_mean,var22_std,var22_skew,var22_kurt,var22_min,var22_1qrt,var22_med,var22_3qrt,var22_max
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.654144,0.000049,3.401137e-12,-1.221011,0.654069,0.654106,0.654144,0.654182,0.654219
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.654165,0.000049,3.426030e-12,-1.254464,0.654092,0.654129,0.654165,0.654202,0.654239
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.654187,0.000049,3.401137e-12,-1.221011,0.654111,0.654149,0.654187,0.654224,0.654262
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.654208,0.000049,5.394783e-12,-1.254464,0.654134,0.654171,0.654208,0.654245,0.654282
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.654229,0.000049,4.374979e-12,-1.221011,0.654154,0.654192,0.654229,0.654267,0.654305


---------

## ⚡ Exponentially Weighted Statistical Feature Extraction


The `ExtractEWStatisticalFeatures` class provides a specialized version of the standard statistical features. The "EW" stands for **Exponentially Weighted**.

In this method, not all data points in a window are treated equally. Instead, more recent data points are given progressively higher weight than older points. The rate at which the importance of older data "decays" is controlled by the `decay` parameter.

This is particularly useful in scenarios where the most recent behavior within a window is more predictive of the outcome than the behavior at the beginning of the window. It creates features that are more sensitive to the latest changes in the signal.

Again, we will use the same `windowed_data` `DataFrame` as input.


In [21]:
# Configure the EW statistical feature extractor
# The decay parameter is specific to this class and controls the weighting
ew_statistical_config = EWStatisticalConfig(
    window_size=WINDOW_SIZE, 
    overlap=OVERLAP,       
    decay=0.9,             
)

# Instantiate the extractor
ew_feature_extractor = ExtractEWStatisticalFeatures(ew_statistical_config)
ew_feature_extractor.is_windowed = True

# Run the feature extraction step by calling the instance
ew_statistical_features = ew_feature_extractor(windowed_data)

# Display the results
print("Shape of the final EW statistical features:", ew_statistical_features.shape)
print("\nColumns are named 'var<index>_ew_<feature>', and the label column is preserved.")
ew_statistical_features.head()


Shape of the final EW statistical features: (62061, 198)

Columns are named 'var<index>_ew_<feature>', and the label column is preserved.


,var1_ew_mean,var1_ew_std,var1_ew_skew,var1_ew_kurt,var1_ew_min,var1_ew_1qrt,var1_ew_med,var1_ew_3qrt,var1_ew_max,var2_ew_mean,...,var21_ew_max,var22_ew_mean,var22_ew_std,var22_ew_skew,var22_ew_kurt,var22_ew_min,var22_ew_1qrt,var22_ew_med,var22_ew_3qrt,var22_ew_max
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.654156,0.000048,-0.269855,1.744778,-1.769354,-1.002305,-0.235257,0.531791,1.298839
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.654177,0.000048,-0.275534,1.715208,-1.747155,-0.991197,-0.235240,0.520718,1.276676
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.654198,0.000048,-0.269855,1.744778,-1.769353,-1.002305,-0.235257,0.531791,1.298839
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.654219,0.000048,-0.275534,1.715208,-1.747154,-0.991197,-0.235239,0.520718,1.276676
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.654241,0.000048,-0.269854,1.744777,-1.769353,-1.002305,-0.235257,0.531791,1.298840
